In [1]:
%reload_ext autoreload
%autoreload 2
from src.acnets.pipeline import Parcellation
from src.acnets.pipeline import TimeseriesAggregator, ConnectivityExtractor
from src.acnets.pipeline import ConnectivityAggregator
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline

In [26]:
# parcellation
parcellation = Parcellation(atlas_name='dosenbach2010')
dataset = parcellation.fit_transform(None)

# time-series aggregation
ts_agg = TimeseriesAggregator(strategy=None)
dataset = ts_agg.fit_transform(dataset)

# connectivity extraction
conn_extractor = ConnectivityExtractor(kind='correlation')
dataset = conn_extractor.fit_transform(dataset)

# connectivity aggregation
conn_agg = ConnectivityAggregator(strategy='network')
dataset = conn_agg.fit_transform(dataset)

# TODO: just mean/std of parcellation timeseries (160 features)
dataset['features_regions'] = dataset['timeseries'].mean('timepoint')

# # TODO: only diagonal of tagg-None_cagg-Network (6 features)
dataset['features_within_networks'] = (('subject', 'network_src'), np.array([np.diag(c) for c in dataset['connectivity']]))

# # TODO: only upper triangle (excluding diagonal) of tagg-Network_cagg-None (15 features)
# TODO_features_between_networks = dataset['connectivity']

dataset

<xarray.Dataset>
Dimensions:                   (timepoint: 124, region: 160, subject: 32,
                               region_src: 160, region_dst: 160,
                               network_src: 6, network_dst: 6)
Coordinates:
  * timepoint                 (timepoint) int64 1 2 3 4 5 ... 121 122 123 124
  * region                    (region) object 'vmPFC 1' ... 'post occipital 160'
  * subject                   (subject) object 'AVGP01' 'AVGP02' ... 'NVGP19NEW'
    network                   (region) object 'default' ... 'occipital'
  * region_src                (region_src) object 'vmPFC 1' ... 'post occipit...
  * region_dst                (region_dst) object 'vmPFC 1' ... 'post occipit...
  * network_src               (network_src) object 'cerebellum' ... 'sensorim...
  * network_dst               (network_dst) object 'cerebellum' ... 'sensorim...
Data variables: (12/14)
    timeseries                (subject, timepoint, region) float64 259.0 ... ...
    age                       (subject) float64 ...
    sex                       (subject) object ...
    group                     (subject) object ...
    preprocessed_beh          (subject) float64 ...
    preprocessed_rsfmri       (subject) float64 ...
    ...                        ...
    y                         (region) int64 ...
    z                         (region) int64 ...
    index                     (region) int64 ...
    connectivity              (subject, network_src, network_dst) float64 0.5...
    features_regions          (subject, region) float64 255.8 596.0 ... 605.4
    features_within_networks  (subject, network_src) float64 0.5311 ... 0.3629

In [ ]:
debug_pipe = Pipeline([
    ('parcellation', Parcellation('dosenbach2010')),
    ('timeseries_aggregation', TimeseriesAggregator(strategy=None)),
    ('connectivity', ConnectivityExtractor('partial correlation')),
    ('connectivity_aggregation', ConnectivityAggregator(strategy='network')),
])

debug_pipe.fit_transform(None)  # load data for all participants
